In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc2_preprocessor import Preprocessor

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Fit complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
Transform complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
Transform complete.


## Vanilla CatBoost

In [8]:
cb_clf = cb.CatBoostClassifier(
    objective='Logloss',
    random_state=42,
    thread_count=-1
)

In [9]:
cat_feature_names = pre.cat_for_encoding_
cb_clf.fit(X_train_proc, y_train, 
           cat_features=cat_feature_names,
           verbose=False)

In [10]:
test_probabilities = cb_clf.predict_proba(X_test_proc)[:, 1]

test_classes = cb_clf.predict(X_test_proc)

print(f"Accuracy: {accuracy_score(y_test, test_classes)}")
print(f"F1 Score: {f1_score(y_test, test_classes)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, test_probabilities)}") # Use probabilities
print(f"PR AUC (Average Precision): {average_precision_score(y_test, test_probabilities)}") # Use probabilities
print(f"Precision: {precision_score(y_test, test_classes)}")
print(f"Recall: {recall_score(y_test, test_classes)}")

Accuracy: 0.8112962962962963
F1 Score: 0.5108017282765243
ROC AUC Score: 0.8344394923961488
PR AUC (Average Precision): 0.6043784760678073
Precision: 0.6273584905660378
Recall: 0.4307692307692308


## Optuna

In [11]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'witness_present_ind']


In [13]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [14]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=50, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-06 16:47:52,020] A new study created in memory with name: no-name-e38be2ca-1a82-43b5-8c1b-ac2e23caf0b4



2. Starting Optuna study...


Best trial: 0. Best value: 0.576561:   2%|▏         | 1/50 [00:05<04:07,  5.06s/it]

[I 2025-11-06 16:47:57,085] Trial 0 finished with value: 0.5765606595995288 and parameters: {'learning_rate': 0.0146154791390475, 'depth': 4, 'l2_leaf_reg': 0.1444278477876349, 'subsample': 0.7, 'random_strength': 5.662041462785675e-07, 'bagging_temperature': 0.20885984194780416, 'border_count': 132, 'scale_pos_weight': 3.551127359417357}. Best is trial 0 with value: 0.5765606595995288.


Best trial: 0. Best value: 0.576561:   4%|▍         | 2/50 [00:10<04:24,  5.52s/it]

...

[I 2025-11-06 16:50:31,692] Trial 47 finished with value: 0.5780380673499268 and parameters: {'learning_rate': 0.011595950942832214, 'depth': 8, 'l2_leaf_reg': 0.027325055485873037, 'subsample': 0.7, 'random_strength': 3.874100931892773e-07, 'bagging_temperature': 0.5047741010335989, 'border_count': 167, 'scale_pos_weight': 4.017586044443871}. Best is trial 21 with value: 0.5945757997218358.


Best trial: 21. Best value: 0.594576:  98%|█████████▊| 49/50 [02:45<00:04,  4.72s/it]

[I 2025-11-06 16:50:37,137] Trial 48 finished with value: 0.586093920640699 and parameters: {'learning_rate': 0.0135174129572402, 'depth': 7, 'l2_leaf_reg': 0.001031104676216177, 'subsample': 0.9, 'random_strength': 0.0002823271490907119, 'bagging_temperature': 0.1456290316067929, 'border_count': 224, 'scale_pos_weight': 2.1222076960335334}. Best is trial 21 with value: 0.5945757997218358.


Best trial: 21. Best value: 0.594576: 100%|██████████| 50/50 [02:49<00:00,  3.40s/it]

[I 2025-11-06 16:50:41,783] Trial 49 finished with value: 0.5865771812080537 and parameters: {'learning_rate': 0.017268833788154022, 'depth': 6, 'l2_leaf_reg': 0.16951070830968767, 'subsample': 0.8, 'random_strength': 0.08480734016786749, 'bagging_temperature': 0.6453535890663101, 'border_count': 180, 'scale_pos_weight': 2.5728435639534597}. Best is trial 21 with value: 0.5945757997218358.

Optuna study finished.
Number of finished trials: 50

Best trial:
  Value (Max F1 Score): 0.5946
  Best Hyperparameters:
    learning_rate: 0.028990278077965104
    depth: 5
    l2_leaf_reg: 0.046395270105008594
    subsample: 0.8
    random_strength: 0.0024402603250790666
    bagging_temperature: 0.046374718040806784
    border_count: 111
    scale_pos_weight: 2.3200395172890085


In [15]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 50 # Keep early stopping
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.028990278077965104, 'depth': 5, 'l2_leaf_reg': 0.046395270105008594, 'subsample': 0.8, 'random_strength': 0.0024402603250790666, 'bagging_temperature': 0.046374718040806784, 'border_count': 111, 'scale_pos_weight': 2.3200395172890085}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5946

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.90      0.81      0.85      4165
   Class 1.0       0.52      0.69      0.59      1235

    accuracy                           0.78      5400
   macro avg       0.71      0.75      0.72      5400
weighted avg       0.81      0.78      0.79      5400

